In [1]:
import numpy as np

In [127]:
names= ['x', 'y', 'z', 'CHOL', 'DPPC', 'DIPC', 'Head', 'Tail']  
for i in range(12):
    temp = 'BL'+str(i+1)
    names.append(temp)  

print "Feature names:\n", names

Feature names:
['x', 'y', 'z', 'CHOL', 'DPPC', 'DIPC', 'Head', 'Tail', 'BL1', 'BL2', 'BL3', 'BL4', 'BL5', 'BL6', 'BL7', 'BL8', 'BL9', 'BL10', 'BL11', 'BL12']


In [140]:
x = np.zeros([100,3040,12], dtype={'names':names, 'formats':['float']*len(names)})

In [141]:
test = x[0,0,:]
test.shape

(12,)

In [142]:

test = x[['BL1', 'BL2']]
x['BL1'] = np.random.random(size=x['BL1'].shape)


In [143]:
x[3,5,2] = np.random.random(size=(20))

In [144]:
x[3,5,2]

( 0.4983349,  0.23191608,  0.60208335,  0.91894745,  0.09907514,  0.19367652,  0.12869451,  0.52786714,  0.24513069,  0.9311619,  0.43514899,  0.02051529,  0.523117,  0.36231273,  0.96497782,  0.91572776,  0.69055924,  0.04302336,  0.96235278,  0.81439592)

In [1]:
import glob 

In [4]:
test = glob.glob('Pilot2/common/generate_datasets/*.npz')

In [14]:
for i in test:
    print '/Users/karande1/Benchmarks/' + i.split('/')[-1].split('.npz')[0] + '_AE_Include_Conv.npy'

/Users/karande1/Benchmarks/test_01_outof_29_AE_Include_Conv.npy
/Users/karande1/Benchmarks/test_02_outof_29_AE_Include_Conv.npy
/Users/karande1/Benchmarks/test_03_outof_29_AE_Include_Conv.npy
/Users/karande1/Benchmarks/test_16_outof_29_AE_Include_Conv.npy
/Users/karande1/Benchmarks/test_17_outof_29_AE_Include_Conv.npy
/Users/karande1/Benchmarks/test_18_outof_29_AE_Include_Conv.npy


In [8]:
%pwd

u'/Users/karande1/Benchmarks'